In [1]:
import pandas as pd
from sklearn.model_selection import ParameterGrid
from neuralprophet import NeuralProphet, set_log_level, set_random_seed
from src.model.features import add_stock_price_feature

# Disable logging messages unless there is an error
set_log_level("ERROR")

df = pd.read_csv('data/stocks/2330_stock_data_0317.csv', parse_dates=['ds'])
df = add_stock_price_feature(df)
# plt = df.plot(x="ds", y="y", figsize=(15, 5))
df.info()

c:\Users\nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


<class 'pandas.core.frame.DataFrame'>
Int64Index: 566 entries, 4 to 569
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Unnamed: 0                566 non-null    int64         
 1   ds                        566 non-null    datetime64[ns]
 2   open_price                566 non-null    float64       
 3   high_price                566 non-null    float64       
 4   low_price                 566 non-null    float64       
 5   y                         566 non-null    float64       
 6   volume                    566 non-null    int64         
 7   foreign                   566 non-null    int64         
 8   investment                566 non-null    int64         
 9   dealer                    566 non-null    int64         
 10  ratio_over_400_shares     566 non-null    float64       
 11  shareholders_400_to_600   566 non-null    int64         
 12  shareholders_600_to_80

In [2]:
from src.model.utils import val_mape

param_grid = {
    'ratio_over_400_shares': [0, 5],
    'shareholders_400_to_600': [0, 5],
    'shareholders_600_to_800': [0, 5],
    'shareholders_800_to_1000': [0, 5],
    'ratio_over_1000_shares': [0, 5],
}

results = []
# Iterate over each combination of hyperparameters
for params in ParameterGrid(param_grid):
    # Initialize the NeuralProphet model with current hyperparameters
    print([params])
    m = NeuralProphet(
        yearly_seasonality=False,
        weekly_seasonality=True,
        n_lags=3,
    )
    m = m.add_country_holidays("TW")
    
    lag_regs = (
        # ('volume', 8),
        # ('high_low_diff', 10),
        ('ratio_over_400_shares', params['ratio_over_400_shares']),
        ('shareholders_400_to_600', params['shareholders_400_to_600']),
        ('shareholders_600_to_800', params['shareholders_600_to_800']),
        ('shareholders_800_to_1000', params['shareholders_800_to_1000']),
        ('ratio_over_1000_shares', params['ratio_over_1000_shares']),
    )
    columes = ['ds', 'y']
    for col, lag in lag_regs:
        if lag > 0:
            m.add_lagged_regressor(col, n_lags=lag)
            columes.append(col)
    
    df_train, df_val = m.split_df(df[columes], valid_p=0.2)
    set_random_seed(0)
    metrics = m.fit(df_train, validation_df=df_val)
    
    # Create a new dataframe reaching 365 into the future for our forecast, n_historic_predictions also shows historic data
    df_future = m.make_future_dataframe(df[columes], n_historic_predictions=True, periods=7)

    # Predict the future
    forecast = m.predict(df_future)
    
    rmse = metrics.iloc[-1]['RMSE_val']
    mape = val_mape(df_val, forecast)
    results.append({**params, 'RMSE': rmse, 'MAPE': mape})

[{'ratio_over_1000_shares': 0, 'ratio_over_400_shares': 0, 'shareholders_400_to_600': 0, 'shareholders_600_to_800': 0, 'shareholders_800_to_1000': 0}]
Training: |          | 0/? [00:00<?, ?it/s]

WARNING - (py.warnings._showwarnmsg) - c:\Users\nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\utilities\data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 32. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.

WARNING - (py.warnings._showwarnmsg) - c:\Users\nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\utilities\data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 11. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.

Finding best initial lr: 100%|██████████| 219/219 [00:00<00:00, 265.73it/s]


Training: |          | 0/? [00:09<?, ?it/s, v_num=1634, MAE_val=14.80, RMSE_val=18.10, Loss_val=0.00162, RegLoss_val=0.000, train_loss=0.000691, reg_loss=0.000, MAE=8.240, RMSE=11.90, Loss=0.000666, RegLoss=0.000]
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 284.88it/s]
[{'ratio_over_1000_shares': 0, 'ratio_over_400_shares': 0, 'shareholders_400_to_600': 0, 'shareholders_600_to_800': 0, 'shareholders_800_to_1000': 5}]
Training: |          | 0/? [00:00<?, ?it/s]

WARNING - (py.warnings._showwarnmsg) - c:\Users\nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\utilities\data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 9. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.

Finding best initial lr: 100%|██████████| 219/219 [00:00<00:00, 277.04it/s]


Training: |          | 0/? [00:09<?, ?it/s, v_num=1635, MAE_val=15.00, RMSE_val=18.10, Loss_val=0.00164, RegLoss_val=0.000, train_loss=0.000678, reg_loss=0.000, MAE=8.420, RMSE=11.90, Loss=0.000674, RegLoss=0.000]
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 312.52it/s]
[{'ratio_over_1000_shares': 0, 'ratio_over_400_shares': 0, 'shareholders_400_to_600': 0, 'shareholders_600_to_800': 5, 'shareholders_800_to_1000': 0}]
Training: |          | 0/? [00:00<?, ?it/s]

...

Training: |          | 0/? [00:09<?, ?it/s, v_num=1663, MAE_val=17.20, RMSE_val=21.30, Loss_val=0.00227, RegLoss_val=0.000, train_loss=0.000754, reg_loss=0.000, MAE=9.140, RMSE=12.60, Loss=0.000724, RegLoss=0.000]
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 308.29it/s]
[{'ratio_over_1000_shares': 5, 'ratio_over_400_shares': 5, 'shareholders_400_to_600': 5, 'shareholders_600_to_800': 5, 'shareholders_800_to_1000': 0}]
Training: |          | 0/? [00:00<?, ?it/s]

Finding best initial lr: 100%|██████████| 219/219 [00:00<00:00, 264.13it/s]


Training: |          | 0/? [00:10<?, ?it/s, v_num=1664, MAE_val=16.30, RMSE_val=20.50, Loss_val=0.00209, RegLoss_val=0.000, train_loss=0.000789, reg_loss=0.000, MAE=9.400, RMSE=12.90, Loss=0.000759, RegLoss=0.000]
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 272.94it/s]
[{'ratio_over_1000_shares': 5, 'ratio_over_400_shares': 5, 'shareholders_400_to_600': 5, 'shareholders_600_to_800': 5, 'shareholders_800_to_1000': 5}]
Training: |          | 0/? [00:00<?, ?it/s]

Finding best initial lr: 100%|██████████| 219/219 [00:00<00:00, 270.46it/s]


Training: |          | 0/? [00:10<?, ?it/s, v_num=1665, MAE_val=17.00, RMSE_val=20.70, Loss_val=0.00213, RegLoss_val=0.000, train_loss=0.000881, reg_loss=0.000, MAE=10.30, RMSE=14.30, Loss=0.000926, RegLoss=0.000]
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 285.06it/s]


In [3]:
results_df = pd.DataFrame(results)
results_df.to_csv('params.csv')
results_df

,ratio_over_1000_shares,ratio_over_400_shares,shareholders_400_to_600,shareholders_600_to_800,shareholders_800_to_1000,RMSE,MAPE
0,0,0,0,0,0,18.057192,0.013987
1,0,0,0,0,5,18.149027,0.014366
2,0,0,0,5,0,18.789856,0.015052
3,0,0,0,5,5,17.737442,0.014084
4,0,0,5,0,0,17.784914,0.013899
5,0,0,5,0,5,17.493376,0.013677
6,0,0,5,5,0,17.572212,0.013501
7,0,0,5,5,5,17.855698,0.013964
8,0,5,0,0,0,18.070894,0.014289
9,0,5,0,0,5,18.397350,0.014393


In [4]:
results_df.sort_values(by="RMSE").head(10)

,ratio_over_1000_shares,ratio_over_400_shares,shareholders_400_to_600,shareholders_600_to_800,shareholders_800_to_1000,RMSE,MAPE
5,0,0,5,0,5,17.493376,0.013677
12,0,5,5,0,0,17.558504,0.013526
6,0,0,5,5,0,17.572212,0.013501
3,0,0,0,5,5,17.737442,0.014084
4,0,0,5,0,0,17.784914,0.013899
10,0,5,0,5,0,17.824684,0.013753
7,0,0,5,5,5,17.855698,0.013964
15,0,5,5,5,5,17.931187,0.014168
16,5,0,0,0,0,17.973949,0.014302
0,0,0,0,0,0,18.057192,0.013987


In [5]:
results_df.sort_values(by="MAPE").head(10)

,ratio_over_1000_shares,ratio_over_400_shares,shareholders_400_to_600,shareholders_600_to_800,shareholders_800_to_1000,RMSE,MAPE
6,0,0,5,5,0,17.572212,0.013501
12,0,5,5,0,0,17.558504,0.013526
5,0,0,5,0,5,17.493376,0.013677
10,0,5,0,5,0,17.824684,0.013753
13,0,5,5,0,5,18.062065,0.013891
4,0,0,5,0,0,17.784914,0.013899
7,0,0,5,5,5,17.855698,0.013964
0,0,0,0,0,0,18.057192,0.013987
24,5,5,0,0,0,18.128235,0.014038
3,0,0,0,5,5,17.737442,0.014084
